In [1]:
import json
import pickle

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from indra_db.util.content_scripts import get_text_content_from_pmids
from indra_db.util.content_scripts import get_stmts_with_agent_text_like
from indra_db.util.content_scripts import get_text_content_from_stmt_ids

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator

from adeft_indra.s3 import model_to_s3
from adeft_indra.ground import gilda_ground

In [2]:
shortforms = ['CKI']
genes = ['CHKA']
families = {}
groundings = [f'HGNC:{get_hgnc_id(gene)}' for gene in genes]
for family, members in families.items():
    genes.extend(members)
    groundings.append(f'FPLX:{family}')
with open('../data/entrez_all_pmids.json', 'r') as f:
    all_pmids = json.load(f)

In [3]:
genes

['CHKA']

In [4]:
entrez_texts = []
entrez_refs = set()
for gene, grounding in zip(genes, groundings):
    try:
        pmids = all_pmids[gene]
    except KeyError:
        continue
    _, content = get_text_content_from_pmids(pmids)
    entrez_texts.extend([(universal_extract_text(text), grounding)
                          for text in content.values() if text])
    entrez_refs.update(content.keys())

In [5]:
miners = dict()
all_texts = set()
for shortform in shortforms:
    stmts = get_stmts_with_agent_text_like(shortform)[shortform]
    _, content = get_text_content_from_stmt_ids(stmts)
    shortform_texts = [universal_extract_text(text, contains=shortforms)
                       for ref, text in content.items() if text and ref not in entrez_refs]
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(shortform_texts)
    all_texts.union(shortform_texts)

It's then necessary to check if Acromine produced the correct results. We must fix errors manually

In [6]:
top = miners['CKI'].top()

In [7]:
top

[('casein kinase i', 83.55172413793103),
 ('inhibitor', 49.42857142857143),
 ('cdk inhibitor', 34.51282051282051),
 ('cyclin dependent kinase inhibitor', 29.125),
 ('compound kushen injection', 15.538461538461538),
 ('and casein kinase i', 12.14285714285714),
 ('by casein kinase i', 12.0),
 ('kinase inhibitor', 7.135135135135137),
 ('the cdk inhibitor', 6.444444444444445),
 ('the cyclin dependent kinase inhibitor', 5.0),
 ('a cyclin dependent kinase inhibitor', 4.857142857142858),
 ('cyclin dependent kinase cdk inhibitor', 4.666666666666667),
 ('the casein kinase i', 4.0),
 ('a cdk inhibitor', 3.6),
 ('a casein kinase i', 3.6),
 ('kinase casein kinase i', 3.6),
 ('casein kinase 1', 3.5),
 ('and cyclin dependent kinase inhibitor', 3.2),
 ('cyclin kinase inhibitor', 3.0),
 ('of casein kinase i', 3.0),
 ('of the cdk inhibitor', 3.0),
 ('and cdk inhibitor', 2.5),
 ('chronic kidney injury', 2.0),
 ('encoding casein kinase i', 2.0),
 ('phosphorylation by casein kinase i', 2.0),
 ('such as ca

In [8]:
longforms0 = miners['CKI'].get_longforms()

In [9]:
longforms0

[('casein kinase i', 90, 0.9963380884723683),
 ('cdk inhibitor', 39, 0.9540692954497554),
 ('cyclin dependent kinase inhibitor', 33, 0.8745254981102235),
 ('compound kushen injection', 17, 0.9968857634969037),
 ('casein kinase 1', 5, 0.8491202853770702),
 ('cyclin kinase inhibitor', 4, 0.9545287798561277),
 ('chronic kidney injury', 3, 0.9992003998667001),
 ('chronic kidney insufficiency', 2, 1.0),
 ('cathepsin k inhibitor', 2, 0.9518236931053591),
 ('cyclin dependent kinase inhibitory', 2, 0.8913012289830017),
 ('checkpoint inhibitor', 2, 0.8762824955766819),
 ('cyclin dependent kinase inhibitory protein', 2, 0.8147856149332774),
 ('casein kinase', 1, 1.0),
 ('casein kinase iδ', 1, 1.0),
 ('cell kinetics index', 1, 1.0),
 ('catk inhibitor', 1, 0.8979244954134231),
 ('casein kinase i family', 1, 0.8913012289830017),
 ('cyclin dependent kinase inhibition', 1, 0.8913012289830017),
 ('cyclin kinase inhibitory protein', 1, 0.8907446064246349),
 ('casein kinase 1α', 1, 0.8502830004171938),


In [13]:
longforms0

[('casein kinase i', 90, 0.9910980477218947),
 ('cdk inhibitor', 39, 0.9615724942397311),
 ('cyclin dependent kinase inhibitor', 33, 0.8860985999085267),
 ('compound kushen injection', 17, 0.996058602011503),
 ('casein kinase 1', 5, 0.8489402220884255),
 ('cyclin kinase inhibitor', 4, 0.9662324204921061),
 ('chronic kidney injury', 3, 0.9991670831944792),
 ('chronic kidney insufficiency', 2, 1.0),
 ('cathepsin k inhibitor', 2, 0.9432704664267384),
 ('cyclin dependent kinase inhibitory', 2, 0.8913012289830017),
 ('checkpoint inhibitor', 2, 0.8677292688980612),
 ('cyclin dependent kinase inhibitory protein', 2, 0.8147856149332774),
 ('casein kinase', 1, 1.0),
 ('casein kinase iδ', 1, 1.0),
 ('cell kinetics index', 1, 1.0),
 ('casein kinase i family', 1, 0.8913012289830017),
 ('cyclin dependent kinase inhibition', 1, 0.8913012289830017),
 ('cyclin kinase inhibitory protein', 1, 0.8907307210073896),
 ('catk inhibitor', 1, 0.8870075381692498),
 ('casein kinase 1α', 1, 0.8502830004171938),
 

In [15]:
from adeft.ground import AdeftGrounder
grounder = AdeftGrounder()

In [16]:
groundings = [(longform, grounder.ground(longform)) for longform, _, _ in longforms0]

In [22]:
groundings = [(longform, grounding if grounding and len(grounding[0]['longform_text'].split()) == len(longform.split())
               else None) for longform, grounding in groundings]

In [23]:
groundings

[('casein kinase i', None),
 ('cdk inhibitor',
  [{'name_space': 'CHEBI',
    'id': 'CHEBI:82665',
    'canonical_name': 'EC 2.7.11.22 (cyclin-dependent kinase) inhibitor',
    'type': 'synonym',
    'raw_text': 'CDK inhibitor',
    'longform_text': 'cdk inhibitor',
    'score': (1.0, 1)}]),
 ('cyclin dependent kinase inhibitor', None),
 ('compound kushen injection', None),
 ('casein kinase 1', None),
 ('cyclin kinase inhibitor', None),
 ('chronic kidney injury', None),
 ('chronic kidney insufficiency',
  [{'name_space': 'MESH',
    'id': 'D007676',
    'canonical_name': 'Kidney Failure, Chronic',
    'type': 'synonym',
    'raw_text': 'Chronic Kidney Insufficiency',
    'longform_text': 'chronic kidney insufficiency',
    'score': (1.0, 1)},
   {'name_space': 'MESH',
    'id': 'D051436',
    'canonical_name': 'Renal Insufficiency, Chronic',
    'type': 'synonym',
    'raw_text': 'Chronic Kidney Insufficiency',
    'longform_text': 'chronic kidney insufficiency',
    'score': (1.0, 1)}

In [36]:
longforms0.extend([(longform, score) for i, (longform, score) in enumerate(top)
                   if i in [2, 3]])

In [37]:
longforms0.sort(key=lambda x: -x[1])

In [38]:
longforms, scores = zip(*longforms0)

In [39]:
longforms

('casein kinase i',
 'cdk inhibitor',
 'cyclin dependent kinase inhibitor',
 'compound kushen injection',
 'casein kinase 1',
 'chronic kidney injury')

In [40]:
grounding_map = {}
names = {}
for longform in longforms:
    grounding = gilda_ground(longform)
    if grounding[0]:
        grounding_map[longform] = f'{grounding[0]}:{grounding[1]}'
        names[grounding_map[longform]] = grounding[2]

In [41]:
grounding_map

{'casein kinase i': 'FPLX:CSNK1',
 'cdk inhibitor': 'CHEBI:CHEBI:82665',
 'casein kinase 1': 'FPLX:CSNK1'}

In [42]:
names

{'FPLX:CSNK1': 'CSNK1',
 'CHEBI:CHEBI:82665': 'EC 2.7.11.22 (cyclin-dependent kinase) inhibitor'}

In [43]:
grounding_map, names, pos_labels = ground_with_gui(longforms, scores, grounding_map=grounding_map, names=names)

In [66]:
names

{'FPLX:CSNK1': 'CSNK1',
 'CHEBI:CHEBI:82665': 'EC 2.7.11.22 (cyclin-dependent kinase) inhibitor'}

In [65]:
grounding_map

{'casein kinase 1': 'FPLX:CSNK1',
 'casein kinase i': 'FPLX:CSNK1',
 'cdk inhibitor': 'CHEBI:CHEBI:82665',
 'chronic kidney injury': 'ungrounded',
 'compound kushen injection': 'ungrounded',
 'cyclin dependent kinase inhibitor': 'CHEBI:CHEBI:82665'}

In [67]:
names['HGNC:1937'] = 'CHKA'
pos_labels.append('HGNC:1937')

In [68]:
result = (grounding_map, names, pos_labels)

In [69]:
result

({'casein kinase 1': 'FPLX:CSNK1',
  'casein kinase i': 'FPLX:CSNK1',
  'cdk inhibitor': 'CHEBI:CHEBI:82665',
  'chronic kidney injury': 'ungrounded',
  'compound kushen injection': 'ungrounded',
  'cyclin dependent kinase inhibitor': 'CHEBI:CHEBI:82665'},
 {'FPLX:CSNK1': 'CSNK1',
  'CHEBI:CHEBI:82665': 'EC 2.7.11.22 (cyclin-dependent kinase) inhibitor',
  'HGNC:1937': 'CHKA'},
 ['CHEBI:CHEBI:82665', 'FPLX:CSNK1', 'HGNC:1937'])

In [70]:
grounding_map, names, pos_labels = ({'casein kinase 1': 'FPLX:CSNK1',
  'casein kinase i': 'FPLX:CSNK1',
  'cdk inhibitor': 'CHEBI:CHEBI:82665',
  'chronic kidney injury': 'ungrounded',
  'compound kushen injection': 'ungrounded',
  'cyclin dependent kinase inhibitor': 'CHEBI:CHEBI:82665'},
 {'FPLX:CSNK1': 'CSNK1',
  'CHEBI:CHEBI:82665': 'EC 2.7.11.22 (cyclin-dependent kinase) inhibitor',
  'HGNC:1937': 'CHKA'},
 ['CHEBI:CHEBI:82665', 'FPLX:CSNK1', 'HGNC:1937'])

In [71]:
grounding_dict = {'CKI': grounding_map}

In [72]:
classifier = AdeftClassifier('CKI', pos_labels=pos_labels)

In [73]:
param_grid = {'C': [100.0], 'max_features': [10000]}

In [74]:
labeler = AdeftLabeler(grounding_dict)

In [75]:
corpus = labeler.build_from_texts(shortform_texts)

In [76]:
corpus.extend(entrez_texts)

In [77]:
texts, labels = zip(*corpus)

In [78]:
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=8)

INFO: [2020-01-16 15:52:55] /Users/albertsteppi/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-01-16 15:52:58] /Users/albertsteppi/adeft/adeft/modeling/classify.py - Best f1 score of 0.9391819099021316 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [79]:
classifier.stats

{'label_distribution': {'CHEBI:CHEBI:82665': 50,
  'FPLX:CSNK1': 82,
  'ungrounded': 14,
  'HGNC:1937': 55},
 'f1': {'mean': 0.9391819099021316, 'std': 0.025434876836646192},
 'precision': {'mean': 0.9317793638227074, 'std': 0.016395459070691332},
 'recall': {'mean': 0.9519203413940256, 'std': 0.03118628662361997}}

In [80]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [81]:
d = disamb.disambiguate(shortform_texts)

In [82]:
a = [text for pred, text in zip(d, shortform_texts)if pred[0].startswith('HGNC')]

In [85]:
a[2]

"Immune checkpoint inhibitors (CKIs) are therapeutic weapons in several advanced malignancies. Performance status is a validated prognostic variable in cancer patients; it possibly affects the efficiency of the immune system. We performed a systematic review and meta-analysis to investigate the predictive role of PS toward treatment with CKIs in cancer patients. Following PRISMA guidelines, an electronic search from PubMed, The Cochrane Library and Embase was performed, from the inception of each database to May 31, 2018. Inclusion criteria were (1) randomized trials comparing CKI with standard therapy for the treatment of patients with solid tumors; (2) information on overall survival (OS) according to PS; (3) full text available; and (4) reported in English language. Data were pooled using HRs for OS according to random effect model. The effect of experimental versus control arms was evaluated in PS\u2009=\u20090 and 1-2 subgroups, and the heterogeneity between the two estimates was 

In [88]:
a[16]

'The OsPCS2 exhibits root- and shoot-specific differential ratios of alternatively spliced transcripts in indica rice under Cd stress, and plays role in Cd and As stress tolerance and accumulation. Enzymatic activity of phytochelatin synthase (PCS) in plant produces phytochelatins, which help in sequestration of heavy metal(loid)s inside the cell vacuole to alleviate toxicity. Here we report that among the two PCS genes-OsPCS1 and OsPCS2 in indica rice (Oryza sativa) cultivar, the OsPCS2 produces an alternatively spliced OsPCS2b transcript that bears the unusual premature termination codon besides the canonically spliced OsPCS2a transcript. Root- and shoot-specific differential ratios of alternatively spliced OsPCS2a and OsPCS2b transcript expressions were observed under cadmium stress. Saccharomyces cerevisiae cells transformed with OsPCS2a exhibited increased cadmium (Cd) and arsenic (As) tolerance and accumulation, unlike the OsPCS2b transformed yeast cells. An intron-containing hai

In [86]:
disamb.dump('CKI', '../results')

In [ ]:
from adeft.disambiguate import load_disambiguator

In [ ]:
d = load_disambiguator('ARG', '../results')

In [ ]:
d.disambiguate(texts[0])

In [ ]:
print(d.info())

In [ ]:
a = load_disambiguator('AR')

In [ ]:
a.disambiguate('Androgen')

In [ ]:
logit = d.classifier.estimator.named_steps['logit']

In [ ]:
logit.classes_

In [87]:
model_to_s3(disamb)